In [1]:
# ! pip install trulens_eval llama_index ftfy regex tqdm git+https://github.com/openai/CLIP.git torch torchvision matplotlib scikit-image qdrant_client

In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
!pip install pdfplumber

In [6]:
import os
import pdfplumber
from llama_index import Document

In [7]:
import json
from llama_index.multi_modal_llms.generic_utils import (
    load_image_urls,
)
from llama_index import SimpleDirectoryReader, Document

# context images
image_path = "nvdi-images"
image_documents = SimpleDirectoryReader(image_path).load_data()

# Function to extract text from a single PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text() + ' '  # Extract text from each page
        return text

# Get a list of PDF file paths from the directory
def get_pdf_file_paths(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.pdf')]

# Your directory containing PDFs
pdf_directory = 'NDVI-analysis'

# Get the list of PDF file paths
pdf_file_paths = get_pdf_file_paths(pdf_directory)

# List to store text documents
text_documents = []

# Iterate over each PDF file path, extract text, and create a Document object
for pdf_path in pdf_file_paths:
    pdf_text = extract_text_from_pdf(pdf_path)
    # Here you might want to split or process the text as per your requirements
    text_documents.append(Document(text=pdf_text))

In [9]:
from llama_index.indices.multi_modal.base import MultiModalVectorStoreIndex
from llama_index.node_parser import SentenceSplitter

node_parser = SentenceSplitter.from_defaults()
image_nodes = node_parser.get_nodes_from_documents(image_documents)
text_nodes = node_parser.get_nodes_from_documents(text_documents)

asl_index = MultiModalVectorStoreIndex(image_nodes + text_nodes)

100%|████████████████████████████████████████| 338M/338M [08:38<00:00, 683kiB/s]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [10]:
from transformers import FuyuProcessor, FuyuForCausalLM


In [12]:
# load model and processor
model_id = "adept/fuyu-8b"
processor = FuyuProcessor.from_pretrained(model_id)
model = FuyuForCausalLM.from_pretrained(model_id, device_map="cuda:0")

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`